In [4]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix

In [3]:
import os
mylist= os.listdir('AudioData/')

['Actor 1', 'Actor 2', 'Actor 3', 'Actor 4']


In [12]:
import librosa
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import numpy as np
from tqdm import tqdm

DATA_PATH = r"C:/Users/Milan/Downloads/Audio_Data_Exercise/AudioData/dataset"



def labels_category(path=DATA_PATH):
    labels = os.listdir(path)
    label_indices = np.arange(0, len(labels))
    return labels, label_indices, to_categorical(label_indices)


def convert(file_path, max_len=11):
    wave, sr = librosa.load(file_path, mono=True, sr=None)
    wave = wave[::3]
    mfcc = librosa.feature.mfcc(wave, sr=16000)

    # If maximum length exceeds mfcc lengths then pad the remaining ones
    if (max_len > mfcc.shape[1]):
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')

    # Else cutoff the remaining parts
    else:
        mfcc = mfcc[:, :max_len]
    
    return mfcc


def array_dir(path=DATA_PATH, max_len=11):
    labels, _, _ = labels_category(path)

    for label in labels:
        # Init mfcc vectors
        mfcc_vectors = []

        wavfiles = [path + '/'+label + '/' + wavfile for wavfile in os.listdir(path + '/' + label)]
        for wavfile in tqdm(wavfiles, "Saving vectors of label - '{}'".format(label)):
            mfcc = convert(wavfile, max_len=max_len)
            mfcc_vectors.append(mfcc)
        np.save(label + '.npy', mfcc_vectors)


def split_data(split_ratio=0.6, random_state=42):
    # Get available labels
    labels, indices, _ = labels_category(DATA_PATH)

    # Getting first arrays
    X = np.load(labels[0] + '.npy')
    y = np.zeros(X.shape[0])

    # Append all of the dataset into one single array, same goes for y
    for i, label in enumerate(labels[1:]):
        x = np.load(label + '.npy')
        X = np.vstack((X, x))
        y = np.append(y, np.full(x.shape[0], fill_value= (i + 1)))

    assert X.shape[0] == len(y)
    ans=train_test_split(X, y, test_size= (1 - split_ratio), random_state=random_state, shuffle=True)
    return ans



def our_dataset(path=DATA_PATH):
    labels, _, _ = labels_category(path)
    data = {}
    for label in labels:
        data[label] = {}
        data[label]['path'] = [path +'/' + label + '/' + wavfile for wavfile in os.listdir(path + '/' + label)]

        vectors = []

        for wavfile in data[label]['path']:
            wave, sr = librosa.load(wavfile, mono=True, sr=None)
            # Downsampling
            wave = wave[::3]
            mfcc = librosa.feature.mfcc(wave, sr=16000)
            vectors.append(mfcc)

        data[label]['mfcc'] = vectors

    return data


def load(path=DATA_PATH):
    data = our_dataset(path)

    dataset = []

    for key in data:
        for mfcc in data[key]['mfcc']:
            dataset.append((key, mfcc))

    return dataset[:100]

In [16]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical
feature_dim_2 = 11
array_dir(max_len=feature_dim_2)
X_train, X_test, y_train, y_test = split_data()
feature_dim_1 = 20
channel = 1
epochs = 100
batch_size = 100
verbose = 1
num_classes = 3

X_train = X_train.reshape(X_train.shape[0], feature_dim_1, feature_dim_2, channel)
X_test = X_test.reshape(X_test.shape[0], feature_dim_1, feature_dim_2, channel)
y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Saving vectors of label - 'sad': 100%|█████████████████████████████████████████████████| 15/15 [00:00<00:00, 63.39it/s]


In [17]:
def get_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(feature_dim_1, feature_dim_2, channel)))
    model.add(Conv2D(48, kernel_size=(2, 2), activation='relu'))
    model.add(Conv2D(120, kernel_size=(2, 2), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    return model

# Predicts one sample
def predict(filepath, model):
    sample = convert(filepath)
    sample_reshaped = sample.reshape(1, feature_dim_1, feature_dim_2, channel)
    return labels_category()[0][
            np.argmax(model.predict(sample_reshaped))
    ]

In [18]:
model = get_model()
model.fit(X_train, y_train_hot, batch_size=batch_size, epochs=epochs, verbose=verbose, validation_data=(X_test, y_test_hot))

Train on 27 samples, validate on 18 samples
Epoch 1/100
27/27 [==============================] - 1s 37ms/step - loss: 4.1683 - acc: 0.2222 - val_loss: 3.5147 - val_acc: 0.3333
Epoch 2/100
27/27 [==============================] - 0s 1ms/step - loss: 3.6239 - acc: 0.3333 - val_loss: 2.8869 - val_acc: 0.3333
Epoch 3/100
27/27 [==============================] - 0s 928us/step - loss: 3.2644 - acc: 0.2963 - val_loss: 1.9571 - val_acc: 0.3333
Epoch 4/100
27/27 [==============================] - 0s 891us/step - loss: 3.8577 - acc: 0.2593 - val_loss: 1.1383 - val_acc: 0.3333
Epoch 5/100
27/27 [==============================] - 0s 891us/step - loss: 1.8374 - acc: 0.4444 - val_loss: 1.2298 - val_acc: 0.3333
Epoch 6/100
27/27 [==============================] - 0s 928us/step - loss: 1.5758 - acc: 0.3333 - val_loss: 1.1765 - val_acc: 0.1111
Epoch 7/100
27/27 [==============================] - 0s 966us/step - loss: 1.2819 - acc: 0.4815 - val_loss: 1.2226 - val_acc: 0.3333
Epoch 8/100
27/27 [=========

In [19]:
print(predict(r'C:\Users\Milan\Downloads\Audio_Data_Exercise\AudioData\dataset\angry\a01.wav', model=model))

angry
